In [33]:
import numpy as np
import pandas as pd

In [34]:
#데이터 불러오기
data = pd.read_csv('final.csv', encoding='UTF-8')
df = pd.DataFrame(data)
print(df)

          시점  자치구역번호   조출생률  고령인구비율  청년 순이동수           사업체수  문화시설 개수  학원수 (개)  \
0     200802       1  10.01     8.4   2244.0  358446.000000     22.0  11261.0   
1     200803       1   9.73     8.4  -5057.0  358339.000000     22.0  11261.0   
2     200804       1   9.40     8.5  -6978.0  358233.000000     22.0  11261.0   
3     200805       1   8.82     8.5  -8537.0  358126.000000     22.0  11261.0   
4     200806       1   8.60     8.5  -9643.0  358020.000000     22.0  11261.0   
...      ...     ...    ...     ...      ...            ...      ...      ...   
3222  202310      17   4.90    17.8    -26.0   46261.503034      NaN   1212.0   
3223  202311      17   4.30    17.9   -135.0   46921.650802      NaN   1212.0   
3224  202312      17   3.90    17.9   -438.0   47464.123543      NaN   1212.0   
3225  202401      17   5.20    18.0   -761.0   47992.410855      NaN      NaN   
3226  202402      17   5.20    18.1   -546.0   48055.994207      NaN      NaN   

      수강인원 (명)  소멸위험지수   단계

In [35]:
df.isna().sum()

시점            0
자치구역번호        0
조출생률          1
고령인구비율        1
청년 순이동수       1
사업체수          0
문화시설 개수     231
학원수 (개)      16
수강인원 (명)     16
소멸위험지수      442
단계          442
dtype: int64

In [36]:
df.fillna(method='ffill', inplace=True)
df.isna().sum()

df.head()

C:\Users\toler\AppData\Local\Temp\ipykernel_18048\8256680.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


,시점,자치구역번호,조출생률,고령인구비율,청년 순이동수,사업체수,문화시설 개수,학원수 (개),수강인원 (명),소멸위험지수,단계
0,200802,1,10.01,8.4,2244.0,358446.0,22.0,11261.0,986758.0,2.0452,안정
1,200803,1,9.73,8.4,-5057.0,358339.0,22.0,11261.0,986758.0,2.0452,안정
2,200804,1,9.40,8.5,-6978.0,358233.0,22.0,11261.0,986758.0,2.0452,안정
3,200805,1,8.82,8.5,-8537.0,358126.0,22.0,11261.0,986758.0,2.0452,안정
4,200806,1,8.60,8.5,-9643.0,358020.0,22.0,11261.0,986758.0,2.0452,안정


In [37]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 소멸위험지수를 범주형 변수로 변환 (예시로 3개의 구간으로 나눔)
df['소멸위험지수_범주'] = pd.cut(df['소멸위험지수'], bins=3, labels=[0, 1, 2])

# XGBoost를 이용한 소멸위험지수 예측을 위해 필요한 전처리
X = df.drop(columns=['시점', '자치구역번호', '단계', '소멸위험지수', 'Cluster', '소멸위험지수_범주'])
y = df['소멸위험지수_범주']

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost 모델 학습
xgb_model = xgb.XGBClassifier(random_state=42)
xgb_model.fit(X_train, y_train)

# 예측
y_pred_train = xgb_model.predict(X_train)
y_pred_test = xgb_model.predict(X_test)

# 평가 (정확도)
accuracy_train = accuracy_score(y_train, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred_test)

print(f'Training Accuracy: {accuracy_train}')
print(f'Test Accuracy: {accuracy_test}')

# Feature Importance 확인
feature_importance = xgb_model.feature_importances_
features = X.columns

for i, (feature, importance) in enumerate(zip(features, feature_importance), 1):
    print(f"Feature {i}: {feature}, Importance: {importance}")

ModuleNotFoundError: No module named 'xgboost'